In [234]:
import config as cfg
import requests
import time
import urllib.parse
import hashlib
import hmac
import base64
import seaborn as sns
import json
import matplotlib.pyplot as plt

api_url = "https://api.kraken.com"

api_endpoints = {
    'Assets': '/0/public/Assets',
    'AssetPairs': '/0/public/AssetPairs',
    'Ticker': '/0/public/Ticker',
    'OHLC': '/0/public/OHLC',

    'Balance': '/0/private/Balance',
    'ExtendedBalance': '/0/private/BalanceEx',
    'Ledgers': '/0/private/Ledgers',
    'QueryLedgers': '/0/private/QueryLedgers',
    'TradeVolume': '/0/private/TradeVolume',
}

# Function to get Kraken signature
def get_kraken_signature(urlpath, data, secret):
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode() 

# Function to make Kraken API request
def kraken_request(uri_path, data, api_key, api_sec, headers=None):
    if headers is None:
        headers = {}

    headers['API-Key'] = api_key
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req


api_key = cfg.api_key
api_sec = cfg.api_priv

# Function to generate a nonce
def generate_nonce():
    return str(int(1000 * time.time()))

def grab_ext_bal():
    # Construct the Kraken API request and get the External Balance Information
    resp = kraken_request(api_endpoints['ExtendedBalance'], {
        "nonce": generate_nonce(),
    }, api_key, api_sec)

    # Convert the response JSON to a Python dictionary
    data = resp.json()
    return data

exBalance = grab_ext_bal()
balanceDict = {}
for asset in exBalance['result']:
    if float(exBalance['result'][asset]['balance']) == 0:
        continue
    balance = exBalance['result'][asset]['balance']
    #print(f"{asset} :{balance}")
    #create new dictionary with only non-zero balances
    balanceDict[asset] = balance
print("balanceDict: ", balanceDict)

 
sns.set_theme(style='darkgrid')

# Define seaborn color palette
sns.set_palette('light:#5A9', n_colors=len(balanceDict))



# Extract labels and values
labels = list(balanceDict.keys())
values = list(balanceDict.values())

fig = plt.figure(figsize=(16, 16), dpi=1600)

# Initialize 9 subplots 3x3
fig, ax = plt.subplots(3, 3 , figsize=(12, 12), dpi=1600)
# Plot pie chart

ax[1,1].pie(values, labels=labels, autopct='%1.1f%%')
plt.title('Asset Distribution')
plt.show()

def grab_all_assets():
    # Grab all asset pair names from the Kraken API
    assetPairs = requests.get(api_url + '/0/public/AssetPairs').json()
    assetPairs = assetPairs['result']
    altNames = [details['altname'] for details in assetPairs.values()]
    altNames = list(assetPairs.keys())
    return altNames

assetPairs = grab_all_assets()


ledgerTypes = ['all']
print(','.join(ledgerTypes))
# Construct the Kraken API request and get ledger information
resp = kraken_request(api_endpoints['Ledgers'], {
        "nonce": generate_nonce(),
        "type": 'all',
        "start": 1695728276,
        "end": 1695828276,
        
    }, api_key, api_sec)

    # Convert the response JSON to a Python dictionary
data = resp.json()
print("Ledger Information: ", data)

'''
# Convert to spot pair names
for i in range(len(portfolioPairs)):
    portfolioPairs[i] = portfolioPairs[i] + "USD"
'''


balanceDict:  {'ATOM.F': '5.20928100', 'MSOL': '0.6047700000', 'SOL.F': '1.1884947880', 'ZGBP': '3.8113'}


<Figure size 25600x25600 with 0 Axes>

all
Ledger Information:  {'error': [], 'result': {'count': 0, 'ledger': {}}}


'\ndef grab_portfolio():\n    # Construct the Kraken API request and get the Portfolio Information\n    resp = kraken_request(\'/0/private/Balance\', {\n        "nonce": str(int(1000*time.time()))\n    }, api_key, api_sec)\n\n    # Convert the response JSON to a Python dictionary\n    data = resp.json()\n    return data\n\ndata = grab_portfolio()\nprint("fresh grab:", data)\nportfolioPairs = data[\'result\'].keys() if data[\'result\'].values() != 0 else next\nportfolioPairs = list(portfolioPairs)\nportfolioDict = data[\'result\']\n\n# Convert to spot pair names\nfor i in range(len(portfolioPairs)):\n    portfolioPairs[i] = portfolioPairs[i] + "USD"\n\n\n# Print the contents of portfolioPairs and assetPairs\nprint("portfolioPairs: ", portfolioPairs)\nprint("assetPairs: ", assetPairs)\n\nrelevantPairs = list(set(portfolioPairs).intersection(assetPairs))\n#uodate portfolioDict to only include relevant pairs\nfor key in list(portfolioDict.keys()):\n    if key not in relevantPairs:\n       

In [ ]:
api_endpoints = {
    'Assets': '/public/Assets',
    'AssetPairs': '/public/AssetPairs',
    'Ticker': '/public/Ticker',
    'OHLC': '/public/OHLC',

    'Balance': '/private/Balance',
    'ExtendedBalance': '/private/BalanceEx',
    'Ledgers': '/private/Ledgers',
    'QueryLedgers': '/private/QueryLedgers',
    'TradeVolume': '/private/TradeVolume',
}

print("api_endpoints: ", api_url + api_endpoints['Assets'])

api_endpoints:  https://api.kraken.com/public/Assets


In [188]:
import requests
import json

url = "https://api.kraken.com/0/private/QueryLedgers"

payload = json.dumps({
  "nonce": 1695828490,
  "id": "LBRWPL-NM5KL-YLWZNU, LBTD5Q-H23DC-YRUPVY",
  "trade": False
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'API-Key': 'jUctxyUKBrXM4MqGIbhtjLGZVG0zs3pJSmdX7IDZRF44SmUZwVGr+MY4',
  'API-Sign': 'FIZ/RPuw4ZCDujZ7m8G0Ige+WwfUyGxTSchO73goSmPtIe4ZSABakZpFQy3VTLpqKqo5aaqXyaQmi+5jymk26A=='
}

response = requests.request("POST", url, headers=headers, data=payload).json()

print(response['result'])

{}
